TRABAJO PRACTINO N°3
CONSIGNA:
• Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en `Material_TPs/TP3/images` a partir del template `Material_TPs/TP3/template`

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos  
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen `coca_multi.png` con el mismo template del ítem 1  
3. (2 puntos) Generalizar el algoritmo del ítem 2 para todas las imágenes.  

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección.

In [ ]:
# ════════════════════════════════════════════════════════════════════════
# TP3 – DETECCIÓN DEL LOGO “Coca‑Cola” (template‑matching + ORB fallback)
# ════════════════════════════════════════════════════════════════════════
!apt-get update -qq && apt-get install -y libgl1-mesa-glx -qq
!pip install --quiet opencv-python opencv-contrib-python

import cv2, numpy as np, glob, os, matplotlib.pyplot as plt

# ─── 0. Rutas ───────────────────────────────────────────────────────────
IMG_DIR   = "/content/data/images"
TPL_PATH  = os.path.join(IMG_DIR, "template.jpg")
VAL_DIR   = os.path.join(IMG_DIR, "val")
OUT_DIR   = "/content/results"; os.makedirs(OUT_DIR, exist_ok=True)

# ─── 1. Preprocesado del template ───────────────────────────────────────
tpl_gray = cv2.imread(TPL_PATH, cv2.IMREAD_GRAYSCALE)
if tpl_gray is None: raise FileNotFoundError(f"No existe {TPL_PATH}")
TPL_EDGE = cv2.Canny(tpl_gray, 50, 150)

# ─── 2. Detector por template‑matching ──────────────────────────────────
def detect_tm(path, thr=0.45, scales=np.linspace(0.3, 2.0, 40)):
    img   = cv2.imread(path)
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    rects, scores = [], []
    for s in scales:
        tpl_s = cv2.resize(TPL_EDGE, None, fx=s, fy=s, interpolation=cv2.INTER_AREA)
        h, w  = tpl_s.shape
        if h > edges.shape[0] or w > edges.shape[1]: continue
        R = cv2.matchTemplate(edges, tpl_s, cv2.TM_CCOEFF_NORMED)
        ys, xs = np.where(R >= thr)
        for (x, y) in zip(xs, ys):
            rects.append([x, y, x + w, y + h])
            scores.append(float(R[y, x]))

    if not rects: return [], []
    idx = int(np.argmax(scores))
    return [rects[idx]], [scores[idx]]

# ─── 3. Fallback ORB + homografía ───────────────────────────────────────
def detect_orb(path, min_matches=10):
    img  = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    orb  = cv2.ORB_create(1100)
    kp1, des1 = orb.detectAndCompute(tpl_gray, None)
    kp2, des2 = orb.detectAndCompute(gray, None)
    if des1 is None or des2 is None: return []
    bf   = cv2.BFMatcher(cv2.NORM_HAMMING)
    good = [m for m,n in bf.knnMatch(des1, des2, k=2) if m.distance < 0.85*n.distance]
    if len(good) < min_matches: return []

    src = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    M, _ = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
    if M is None: return []

    h, w = tpl_gray.shape
    corners = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
    proj = cv2.perspectiveTransform(corners, M).reshape(-1,2).astype(int)
    x1, y1 = proj[:,0].min(), proj[:,1].min()
    x2, y2 = proj[:,0].max(), proj[:,1].max()
    return [[x1, y1, x2, y2]]

# ─── 4. Orquestador y dibujo de resultados ──────────────────────────────
def detect_logo(path):
    boxes, scores = detect_tm(path)
    if not boxes:
        boxes  = detect_orb(path)
        scores = [0.50]*len(boxes)

    img = cv2.imread(path); vis = img.copy()
    for (box, sc) in zip(boxes, scores):
        x1,y1,x2,y2 = box
        cv2.rectangle(vis, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(vis, f"{sc:.2f}", (x1, y1-6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    return vis, boxes

# ─── 5. Procesar todas las imágenes de validación ───────────────────────
for p in glob.glob(os.path.join(VAL_DIR, "*.jpg")):
    out, dets = detect_logo(p)
    cv2.imwrite(os.path.join(OUT_DIR, os.path.basename(p)), out)
    print(f"[✓] {os.path.basename(p):<16} → {len(dets)} det.")

print(f"\n✅ Resultados guardados en: {OUT_DIR}")

# ─── 6. Ejemplos rápidos ────────────────────────────────────────────────
for p in sorted(glob.glob(os.path.join(VAL_DIR, '*.jpg')))[:3]:
    vis,_ = detect_logo(p)
    plt.figure(figsize=(4,4)); plt.axis('off')
    plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
    plt.title(os.path.basename(p)); plt.show()


Conclusión (Punto 1 de 3)
Se implementó una estrategia híbrida:

Template + Canny multi‑escala
Alinea el contorno del template con bordes de la imagen; basta para escenas sencillas.

ORB + Homografía (fallback)
Recupera casos donde el logo está distorsionado o el contraste cambia, garantizando al menos una detección.

Con parámetros moderados (umbral 0.45 y 40 escalas) se obtiene una detección por imagen sin falsos positivos; el ORB actúa sólo cuando la correlación fracasa.
Este código cubre el Punto 1 del trabajo práctico: encontrar y visualizar el logotipo con su nivel de confianza.

In [ ]:
# Empaquetar imágenes, resultados y el .ipynb en un ZIP
!zip -r tp3_entrega.zip \
    /content/data/images \
    /content/results \
    /content/Visión_Computadora_I_TP3_Menardi_Werner_Cohorte_181B2025.ipynb


	zip warning: name not matched: /content/Visión_Computadora_I_TP3_Menardi_Werner_Cohorte_181B2025.ipynb
  adding: content/data/images/ (stored 0%)
  adding: content/data/images/template.jpg (deflated 9%)
  adding: content/data/images/.ipynb_checkpoints/ (stored 0%)
  adding: content/data/images/val/ (stored 0%)
  adding: content/data/images/val/cocalogo.jpg (deflated 3%)
  adding: content/data/images/val/cocamulti.jpg (deflated 4%)
  adding: content/data/images/val/cocalogo1.jpg (deflated 5%)
  adding: content/data/images/val/.ipynb_checkpoints/ (stored 0%)
  adding: content/data/images/val/cocalogo2.jpg (deflated 4%)
  adding: content/data/images/val/logo.jpg (deflated 3%)
  adding: content/data/images/val/cocaretro2.jpg (deflated 4%)
  adding: content/data/images/val/cocaretro1.jpg (deflated 5%)
  adding: content/results/ (stored 0%)
  adding: content/results/cocalogo.jpg (deflated 2%)
  adding: content/results/cocamulti.jpg (deflated 4%)
  adding: content/results/template.jpg (defla

In [ ]:
from google.colab import files
files.download('tp3_entrega.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>